In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [74]:
batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [75]:
torch.manual_seed(1337)


In [4]:
with open("shakespeare.txt", 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [6]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [7]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [8]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [76]:
class BigramLanguageModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T)) # (T,C)
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B,T, vocab_size)

        if targets is None:
            loss =None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [77]:
class Head(nn.Module):
    def __init__(self, head_size) -> None:
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B,T,C) @ (B,C,T) -> (B,T,T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

In [78]:
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size) -> None:
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

In [79]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [80]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head) -> None:
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [81]:
model = BigramLanguageModel()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [82]:
for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

KeyboardInterrupt: 

In [68]:
context = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))


That thow your givin old did was't,
Then you say, sirs o'll our iprace I am barces it; mome?

KING LIET:
Willither:
Northat have more; seee! so loved, Bying eather, boty
But thy qunes amony,
Well's, the lail draither con well's bleselved
to nighty, but the diars thre it here you redumphirred have the kivisive;
Bughts
to
cent so make bust:
Owf if tis yorrous begroness our brack.
O, sheave plyselfing iflown,
Ither back, oulk to give? Should 'readcled currt, enough know
THBRIZABRLYCUS:
In laozetor 


In [14]:
head_size = 16
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)
key = torch.nn.Linear(C, head_size, bias=False)
query = torch.nn.Linear(C, head_size, bias=False)
value = torch.nn.Linear(C, head_size, bias=False)
k = key(x)
q = query(x)
v = value(x)
wei = q @ k.transpose(-2, -1) * head_size**-0.5
wei.shape

torch.Size([4, 8, 8])

In [15]:
wei

tensor([[[ 0.1166, -0.0955, -0.3026, -0.4085,  0.4371, -0.2397,  0.1177,
           0.2409],
         [-0.0303,  0.1114,  0.1870, -0.0177,  0.1577, -0.2026, -0.2033,
           0.0029],
         [-0.7616,  0.4531,  0.0553, -0.9026, -0.5004, -0.1125, -0.0714,
           0.5580],
         [ 0.7269, -0.1801,  0.0625, -0.2449, -0.2098, -0.1545,  0.2998,
           0.1847],
         [-0.0611,  0.0689,  0.3947,  0.3721, -0.2123,  0.5355,  0.0298,
          -0.3774],
         [ 0.6328, -0.1136,  0.0784,  0.4430,  0.3127, -0.2757, -0.4150,
          -0.4023],
         [-0.5575,  0.2546,  0.2817, -0.3413, -0.2358,  0.3059, -0.1807,
          -0.1134],
         [-0.3322,  0.0503, -0.2747, -0.3106,  0.1147, -0.1703,  0.0353,
           0.3376]],

        [[ 0.4613, -0.4934, -0.3315, -0.6173,  0.3560,  0.1786, -0.0016,
           0.7506],
         [-0.5028,  0.1789,  0.0864,  0.0120,  0.2491, -0.2420, -0.4608,
          -0.1596],
         [-0.4066, -0.1980,  0.0604, -0.2523,  0.1077,  0.0017, -0.2

In [16]:
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = torch.nn.functional.softmax(wei, dim=-1)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [17]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4646, 0.5354, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1508, 0.5080, 0.3413, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4354, 0.1758, 0.2241, 0.1648, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1634, 0.1861, 0.2578, 0.2521, 0.1405, 0.0000, 0.0000, 0.0000],
         [0.2497, 0.1184, 0.1434, 0.2065, 0.1813, 0.1007, 0.0000, 0.0000],
         [0.0832, 0.1875, 0.1926, 0.1033, 0.1148, 0.1973, 0.1213, 0.0000],
         [0.0936, 0.1372, 0.0991, 0.0956, 0.1463, 0.1100, 0.1352, 0.1829]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3359, 0.6641, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2613, 0.3219, 0.4168, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3316, 0.4173, 0.0932, 0.1579, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1923, 0.1418, 0.3284, 0.1623, 0.1751, 0.0000, 0.0000, 0.0000],
         [0.1899, 0.087

In [56]:
class BatchNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
    def __call__(self, x):
        xmean = x.mean(1, keepdim=True)
        xvar = x.var(1, keepdim=True)
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        return self.out

torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32, 100)
x = module(x)
x.shape


torch.Size([32, 100])

In [57]:
x[:,0].mean(), x[:, 0].std()

(tensor(0.1469), tensor(0.8803))

In [58]:
x[0,:].mean(), x[0, :].std()

(tensor(-9.5367e-09), tensor(1.0000))